### 출력 파서

LLM이 생성 출력 결과를 필요에 맞게 가공, 구조화

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

gemini_api_key = os.getenv("GEMINI_API_KEY")

In [2]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [3]:
output_parser

StrOutputParser()

In [ ]:
output_parser.invoke("테스트") # runnable

'테스트'

In [6]:
from langchain_core.prompts import PromptTemplate

template = "{language} 할 수 있어?"

prompt = PromptTemplate.from_template(template)

In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    google_api_key=gemini_api_key
)

In [8]:
chain = prompt | llm | output_parser # 체인 구성

In [9]:
chain.invoke({"language": "한국어"})

'네, 한국어 할 수 있습니다. 어떤 질문이든 한국어로 답변해 드릴 수 있습니다!'

### 영어 회화 예제

In [12]:
template = """
당신은 영어를 아주 쉽게 가르치는 친절한 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.
양식은 [FORMAT]를 참고해 주세요.

# 상황:
{question}

# FORMAT:
- 영어 회화:
- 한글 해석:
"""

In [13]:
prompt = PromptTemplate.from_template(template)

In [14]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.1,
    google_api_key=gemini_api_key
)

In [15]:
output_parser = StrOutputParser()

In [16]:
chain = prompt | llm | output_parser

In [22]:
result = chain.invoke({"question": "저는 식당에서 음식을 주문하고 싶어요."})

In [23]:
print(result)

안녕하세요! 식당에서 음식을 주문하는 상황, 아주 쉽고 친절하게 알려드릴게요.
이 대화만 잘 익혀두면 자신감 있게 주문할 수 있을 거예요!

---

**등장인물:**
*   **You (손님):** 당신
*   **Waiter (직원):** 식당 직원

---

- 영어 회화:
**You:** Hello! We're ready to order.
**Waiter:** Great! What can I get for you today?
**You:** Hmm... What do you recommend?
**Waiter:** Our special today is the grilled salmon, and the pasta is also very popular.
**You:** Oh, that sounds good, but I think I'll go with the pasta, please. Does it come with a salad?
**Waiter:** Yes, it comes with a small side salad.
**You:** Perfect! And to drink, I'll have a glass of water, please.
**Waiter:** Alright, so that's the pasta with a side salad and a glass of water. Is that everything?
**You:** Yes, that's all. Thank you!
**Waiter:** Great! It'll be out shortly.

- 한글 해석:
**You:** 안녕하세요! 주문할 준비가 됐어요. (주문할게요.)
**Waiter:** 좋아요! 오늘 무엇을 주문하시겠어요?
**You:** 음... 뭘 추천하세요?
**Waiter:** 오늘의 특별 메뉴는 구운 연어이고, 파스타도 아주 인기가 많아요.
**You:** 오, 맛있겠네요, 하지만 저는 파스타로 할게요. 샐러드도 같이 나오나요?
**Waiter:** 네, 작은 사이드 샐러드

### JsonOutputPaser

In [24]:
from langchain_core.output_parsers import JsonOutputParser

parser = JsonOutputParser()

In [25]:
parser

JsonOutputParser()

In [26]:
parser.get_format_instructions()

'Return a JSON object.'

In [27]:
template = """
당신은 영어를 아주 쉽게 가르치는 친절한 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.
양식은 [FORMAT]를 참고해 주세요.

# 상황:
{question}

# FORMAT:
{format_instructions}
"""

In [28]:
prompt = PromptTemplate(
    template=template,
    input_variables=["question"],
    partial_variables={
        "format_instructions": parser.get_format_instructions()
    }
)

In [32]:
chain = prompt | llm | parser
result = chain.invoke({"question": "저는 식당에서 음식을 주문하고 싶어요."})

In [33]:
import pprint

pprint.pprint(result)

{'conversation': [{'line': 'Excuse me.', 'speaker': 'You'},
                  {'line': 'Hello! Are you ready to order?',
                   'speaker': 'Server'},
                  {'line': 'Yes, I am. What do you recommend today?',
                   'speaker': 'You'},
                  {'line': 'Our special today is the grilled salmon with '
                           "vegetables. It's very popular!",
                   'speaker': 'Server'},
                  {'line': "Oh, that sounds good. I'll have the grilled "
                           'salmon, please.',
                   'speaker': 'You'},
                  {'line': 'Excellent choice! And for a drink?',
                   'speaker': 'Server'},
                  {'line': 'Could I get a glass of orange juice, please?',
                   'speaker': 'You'},
                  {'line': 'Certainly. So, one grilled salmon and one orange '
                           'juice. Anything else?',
                   'speaker': 'Server'},
    

### 일관성 있는 형식으로 출력하기 (JsonOutputParser + Pydantic)

In [34]:
from pydantic import BaseModel, Field

# 원하는 JSON 구조를 Pydantic 클래스로 정의
class EnglishConversation(BaseModel):
    title: str = Field(description="상황에 대한 제목")
    dialogue: list = Field(description="상황에 맞는 영어 대화 리스트")
    explanation: str = Field(description="대화에 대한 쉬운 설명")

In [35]:
# JsonOutputParser에 Pydantic 스키마 전달
parser = JsonOutputParser(pydantic_object=EnglishConversation)

In [36]:
# 프롬프트에 스키마 정보 포함
template = """
당신은 영어를 아주 쉽게 가르치는 친절한 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.
출력 양식은 아래 [FORMAT]을 참고해 주세요. 모든 키 이름은 정확히 이 양식과 일치해야 합니다.

# 상황:
{question}

# FORMAT:
{format_instructions}

# 예시:
{{
  "title": "저녁 식사 정하기 (Deciding Dinner)",
  "dialogue": [
    {{"speaker": "A", "text": "What should we have for dinner?"}},
    {{"speaker": "B", "text": "I'm craving some pizza."}}
  ],
  "explanation": "친구와 저녁 메뉴를 정하는 상황입니다. 간단한 표현을 사용했어요."
}}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["question"],
    partial_variables={
        "format_instructions": parser.get_format_instructions()
    },
)

In [37]:
# 체인 구성 및 실행
chain = prompt | llm | parser

In [38]:
result = chain.invoke({"question": "식당에서 주문하는 상황"})

In [39]:
pprint.pprint(result)

{'dialogue': [{'speaker': 'Waiter', 'text': 'Hi! Are you ready to order?'},
              {'speaker': 'Customer',
               'text': "Yes, I am. I'd like the chicken salad, please."},
              {'speaker': 'Waiter',
               'text': 'Okay, one chicken salad. And for a drink?'},
              {'speaker': 'Customer', 'text': 'A glass of water, please.'},
              {'speaker': 'Waiter',
               'text': 'Alright. So, one chicken salad and a glass of water. '
                       'Anything else?'},
              {'speaker': 'Customer',
               'text': "No, that's all for now. Thank you!"}],
 'explanation': "식당에서 웨이터에게 음식을 주문하는 상황입니다. 'I'd like ~'는 '저는 ~을 원해요'라는 아주 "
                "공손하고 흔한 표현이에요. 'And for a drink?'는 '음료는요?'라는 뜻으로 간단하게 물어볼 때 "
                '사용합니다.',
 'title': '식당에서 주문하기 (Ordering at a Restaurant)'}
